<a href="https://colab.research.google.com/github/adityakulkarni0910/Airline_Tweet_Sentiment_Analysis/blob/main/Airline_Tweet_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import re
from nltk import WordPunctTokenizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
train = pd.read_csv('train.csv',encoding='latin-1')
test = pd.read_csv('test.csv',encoding='latin-1')

In [ ]:
train.head()

,Id,text,Target
0,1,@USAirways ! THE WORST in customer service. @...,-1
1,2,@united call wait times are over 20 minutes an...,-1
2,3,@JetBlue what's up with the random delay on fl...,-1
3,4,@AmericanAir Good morning! Wondering why my p...,0
4,5,@united UA 746. Pacific Rim and Date Night cut...,-1


In [ ]:
test.head()

,id,text
0,7322,@AmericanAir In car gng to DFW. Pulled over 1h...
1,7323,"@AmericanAir after all, the plane didnÂÃÂªt ..."
2,7324,@SouthwestAir can't believe how many paying cu...
3,7325,@USAirways I can legitimately say that I would...
4,7326,@AmericanAir still no response from AA. great ...


In [ ]:
#train, val = train_test_split(train,test_size=0.2,random_state=0)

#train.shape, val.shape

# Preprocesing

In [ ]:
train['text'] = train['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [ ]:
train['text'] = train['text'].apply(lambda x: re.split('\s[^\!\?\$]\s', str(x))[0])

In [ ]:
train

,Id,text,Target
0,1,@USAirways ! THE WORST in customer service. @...,-1
1,2,@united call wait times are over 20 minutes an...,-1
2,3,@JetBlue what's up with the random delay on fl...,-1
3,4,@AmericanAir Good morning! Wondering why my p...,0
4,5,@united UA 746. Pacific Rim and Date Night cut...,-1
...,...,...,...
7315,7316,@AmericanAir followback,0
7316,7317,@united thanks for the help. Wish the phone re...,1
7317,7318,@usairways the. Worst. Ever. #dca #customerser...,-1
7318,7319,@nrhodes85: look! Another apology. DO NOT FLY ...,-1


In [ ]:
train['text'] = train['text'].apply(lambda x: re.sub('\s+',  ' ', str(x)))

In [ ]:
train['text'] = train['text'].apply(lambda x: re.sub('[\n\t]',' ', str(x)))

In [ ]:
train["text"] = train['text'].str.lower()

# Tokenizer

In [ ]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
train['text']=train['text'].apply(tokenizer.tokenize)

In [ ]:
train['text']

0       [@usairways, !, the, worst, in, customer, serv...
1       [@united, call, wait, times, are, over, 20, mi...
2       [@jetblue, what's, up, with, the, random, dela...
3       [@americanair, good, morning, !, wondering, wh...
4       [@united, ua, 746, ., pacific, rim, and, date,...
                              ...                        
7315                           [@americanair, followback]
7316    [@united, thanks, for, the, help, ., wish, the...
7317    [@usairways, the, ., worst, ., ever, ., #dca, ...
7318    [@nrhodes85, :, look, !, another, apology, ., ...
7319    [@united, you, are, by, far, the, worst, airli...
Name: text, Length: 7320, dtype: object

# Stop words

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stop = set(nltk_stopwords.words('english'))
#", ".join(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train['text'] = train['text'].apply(lambda x: [i for i in x if i not in stop])

In [ ]:
train['text'] = train['text'].apply(lambda x: ' '.join(ele for ele in x))

In [ ]:
train['text']

0       @usairways ! worst customer service . @usairwa...
1       @united call wait times 20 minutes airport wai...
2       @jetblue what's random delay flight 1729 ? chance
3       @americanair good morning ! wondering pre-tsa ...
4       @united ua 746 . pacific rim date night cut . ...
                              ...                        
7315                              @americanair followback
7316    @united thanks help . wish phone reps could ac...
7317    @usairways . worst . ever . #dca #customerservice
7318    @nrhodes85 : look ! another apology . fly @usa...
7319                          @united far worst airline .
Name: text, Length: 7320, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = CountVectorizer()
train_v = vectorize.fit_transform(train['text'])

In [ ]:
train_vtoa = train_v.toarray()
vectorized = pd.DataFrame(train_vtoa)
vectorized.columns = vectorize.get_feature_names()

# Models

In [ ]:
clf = LogisticRegression(max_iter=10000)

clf_param= {}
clf_grid = GridSearchCV(clf, clf_param,cv=10, return_train_score=True)
clf_grid.fit(train_v,train['Target'])

GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=10000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [ ]:
y_pred = clf_grid.predict(vectorized)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(train['Target'], y_pred, average='macro')

0.8341757494748424

In [ ]:
from xgboost import XGBClassifier
xgbc= XGBClassifier(random_state=0)
xgbc_param = {
              
             }
xgbc_grid = GridSearchCV(xgbc, xgbc_param,cv=5, return_train_score=True )
xgbc_grid.fit(train_v,train['Target'])

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [ ]:
#y_pred = xgbc_grid.predict(vectorized)
#f1_score(train['Target'], y_pred2, average='macro')

In [ ]:
test['text'] = test['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
test['text'] = test['text'].apply(lambda x: re.split('\s[^\!\?\$]\s', str(x))[0])
test['text'] = test['text'].apply(lambda x: re.sub('\s+',  ' ', str(x)))
test['text'] = test['text'].apply(lambda x: re.sub('[\n\t]',' ', str(x)))
test["text"] = test['text'].str.lower()
test['text']=test['text'].apply(tokenizer.tokenize)
test['text'] = test['text'].apply(lambda x: [i for i in x if i not in stop])
test['text'] = test['text'].apply(lambda x: ' '.join(ele for ele in x))


In [ ]:
test_v = vectorize.transform(test['text'])
test_vtoa = test_v.toarray()
test_vectorized = pd.DataFrame(test_vtoa)
test_vectorized.columns = vectorize.get_feature_names()

In [ ]:
test['Target'] = clf_grid.predict(test_v)
test.to_csv('clf_predictions1.csv')

In [ ]:
test['Target'] = xgbc_grid.predict(test_v)
test.to_csv('xgb_predictions1.csv')